In [6]:
!pip3 install torch
!pip3 install dgl==0.4.3post2 
!pip3 install dglke
!pip install pykeen==1.0.5

import pandas as pd
import numpy as np
import sys

#sys.path.insert(1, '/Users/ravinschmidl/Desktop/CoV-KGE-master/utils')
drkg_file = "https://zenodo.org/record/4661198/files/KG.tsv?download=1"
df = pd.read_csv(drkg_file, sep="\t", index_col = 0)


You are using pip version 18.1, however version 21.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 18.1, however version 21.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 18.1, however version 21.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


/Users/ravinschmidl/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/ravinschmidl/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
import argparse
import gzip, os, csv
import numpy as np
import random
import time
import pykeen
from pykeen.triples import TriplesFactory
from pykeen.pipeline import pipeline
tf = TriplesFactory(path='https://zenodo.org/record/4661198/files/KG.tsv?download=1')
training, testing = tf.split()
result = pipeline(
    training=training,
    testing=testing,
    model='RotatE',
    training_kwargs=dict(num_epochs=12),  # short epochs for testing - you should go higher
)

In [8]:
df = df.drop(['r_name'], axis = 1)

In [9]:
df

,h,r,t
0,FIBJDTSHOUXTKV-BRHMIFOHSA-N,ddi-interactor-in,QNZCBYKSOIHPEH-UHFFFAOYSA-N
1,QNZCBYKSOIHPEH-UHFFFAOYSA-N,ddi-interactor-in,QNZCBYKSOIHPEH-UHFFFAOYSA-N
2,RUDATBOHQWOJDD-UZVSRGJWSA-N,ddi-interactor-in,QNZCBYKSOIHPEH-UHFFFAOYSA-N
3,GHCZAUBVMUEKKP-GYPHWSFCSA-N,ddi-interactor-in,QNZCBYKSOIHPEH-UHFFFAOYSA-N
4,BHQCQFFYRZLCQQ-OELDTZBJSA-N,ddi-interactor-in,QNZCBYKSOIHPEH-UHFFFAOYSA-N
...,...,...,...
3652,EBI-25475912,interacts_with,P59633
3653,30968,MI:0915,EBI-25475912
3654,4828,MI:0915,P0C6X7-PRO_0000283841
3655,9584,MI:0915,10284


In [10]:
triples = df.values.tolist()


In [12]:
num_triples = len(triples) 
num_triples

10683732

In [15]:
# Please make sure the output directory exist.
seed = np.arange(num_triples)
np.random.shuffle(seed)

train_cnt = int(num_triples * 0.9)
train_set = seed[:train_cnt]
train_set = train_set.tolist()
test_set = seed[train_cnt:].tolist()

print(len(train_set))
print(len(test_set))

print(len(test_set)/len(train_set)*100)
with open("/Users/ravinschmidl/Desktop/CoV-KGE-master/train/kg_train.txt", 'w+') as f:
    for idx in train_set:
        f.writelines("{}\t{}\t{}\n".format(triples[idx][0], triples[idx][1], triples[idx][2]))
        
#with open("/train/kg_valid.txt", 'w+') as f:
  #  for idx in valid_set:
      #  f.writelines("{}\t{}\t{}\n".format(triples[idx][0], triples[idx][1], triples[idx][2]))

with open("/Users/ravinschmidl/Desktop/CoV-KGE-master/train/kg_test.txt", 'w+') as f:
    for idx in test_set:
        f.writelines("{}\t{}\t{}\n".format(triples[idx][0], triples[idx][1], triples[idx][2]))

9615358
1068374
11.111120355581145


In [ ]:
#!DGLBACKEND=pytorch dglke_train --dataset KG --data_path /Users/ravinschmidl/Desktop/CoV-KGE-master/train --data_files kg_train.tsv kg_valid.tsv kg_test.tsv --format 'raw_udd_hrt' --model_name RotatE --batch_size 1024 \
#--neg_sample_size 256 --hidden_dim 400 --gamma 12.0 --lr 0.1 --max_step 100000 --log_interval 1000 --batch_size_eval 16 -adv --regularization_coef 1.00E-07 --test --num_thread 0 --gpu 0 1 2 3 4 5 6 7 --num_proc 8 --neg_sample_size_eval 10000 --async_update


Using backend: pytorch
Reading train triples....
Finished. Read 9615358 train triples.
Reading valid triples....
Finished. Read 534186 valid triples.
Reading test triples....
Finished. Read 534188 test triples.
|Train|: 9615358
random partition 9615358 edges into 8 parts
part 0 has 1201920 edges
part 1 has 1201920 edges
part 2 has 1201920 edges
part 3 has 1201920 edges
part 4 has 1201920 edges
part 5 has 1201920 edges
part 6 has 1201920 edges
part 7 has 1201918 edges
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/dgl/base.py:25: UserWarning: multigraph will be deprecated.DGL will treat all graphs as multigraph in the future.
  warnings.warn(msg, warn_type)
|valid|: 534186
|test|: 534188
Total initialize time 45.128 seconds
[W ParallelNative.cpp:206] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:206] Warning: Can

Using random_state=2499365823 to split TriplesFactory(path="/Users/ravinschmidl/Desktop/CoV-KGE-master/KG.tsv")
Requested ratio[0]=0.800 (equal to size 8546985), but got 1.000 (equal to size 10683316) to ensure that all entities/relations occur in train.
No random seed is specified. Setting to 680772697.
No cuda devices were available. The model runs on CPU


In [35]:
train

,h,r,t
0,FIBJDTSHOUXTKV-BRHMIFOHSA-N,ddi-interactor-in,QNZCBYKSOIHPEH-UHFFFAOYSA-N
2,RUDATBOHQWOJDD-UZVSRGJWSA-N,ddi-interactor-in,QNZCBYKSOIHPEH-UHFFFAOYSA-N
3,GHCZAUBVMUEKKP-GYPHWSFCSA-N,ddi-interactor-in,QNZCBYKSOIHPEH-UHFFFAOYSA-N
4,BHQCQFFYRZLCQQ-OELDTZBJSA-N,ddi-interactor-in,QNZCBYKSOIHPEH-UHFFFAOYSA-N
5,RFDAIACWWDREDC-FRVQLJSFSA-N,ddi-interactor-in,QNZCBYKSOIHPEH-UHFFFAOYSA-N
...,...,...,...
10683725,2677,interacts_with,H01205
10683726,DEEPDCSUWZPGPP-UHFFFAOYSA-N,interacts_with,30835
10683727,EBI-25475912,interacts_with,P59633
10683728,30968,MI:0915,EBI-25475912


In [36]:
result = pipeline(
    training=train,
    testing=test,
    model='TransE',
    training_kwargs=dict(num_epochs=12),  # short epochs for testing - you should go higher
)

No random seed is specified. Setting to 2403716284.
No cuda devices were available. The model runs on CPU


TypeError: Training and testing must both be given as strings or Triples Factories

In [22]:
help(TriplesFactory)

Help on class TriplesFactory in module pykeen.triples.triples_factory:

class TriplesFactory(builtins.object)
 |  TriplesFactory(*, path: Union[NoneType, str, TextIO] = None, triples: Union[numpy.ndarray, NoneType] = None, create_inverse_triples: bool = False, entity_to_id: Union[Mapping[str, int], NoneType] = None, relation_to_id: Union[Mapping[str, int], NoneType] = None, compact_id: bool = True) -> None
 |  
 |  Create instances given the path to triples.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, *, path: Union[NoneType, str, TextIO] = None, triples: Union[numpy.ndarray, NoneType] = None, create_inverse_triples: bool = False, entity_to_id: Union[Mapping[str, int], NoneType] = None, relation_to_id: Union[Mapping[str, int], NoneType] = None, compact_id: bool = True) -> None
 |      Initialize the triples factory.
 |      
 |      :param path: The path to a 3-column TSV file with triples in it. If not specified,
 |       you should specify ``triples``.
 |      :param tripl